### **DAY 11 (19/01/26) – Statistical Analysis & ML Prep**

### Learn:

- Descriptive statistics
- Hypothesis testing
- A/B test design
- Feature engineering

### 🛠️ Tasks:

1. Calculate statistical summaries
2. Test hypotheses (weekday vs weekend)
3. Identify correlations
4. Engineer features for ML

In [0]:
Oct = spark.read.csv('/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv',header=True,inferSchema=True)
Nov = spark.read.csv('/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv',header=True,inferSchema=True)
full_df = Oct.unionAll(Nov)

In [0]:
full_df = full_df.withColumn("event_date",F.to_date('event_time'))
full_df.repartition(500,'event_date')

In [0]:
silver = full_df.filter(F.col('price')>0)\
        .dropDuplicates(['user_session','event_time'])\
        .withColumn("price_tier",
                    F.when(F.col('price') <10,'Budget')\
                     .when(F.col('price') <50,'Mid')\
                     .otherwise('Premium'))
silver.write.saveAsTable("ecommerce_full_data")

In [0]:
events = spark.sql('select * from ecommerce.default.ecommerce_full_data')

In [0]:
events.printSchema()
events.limit(5).display()

In [0]:
# Descriptive stats
events.describe(["price"]).show()

In [0]:
# Hypothesis: weekday vs weekend conversion
from pyspark.sql import functions as F

events_new=events.withColumn(
    'is_weekend',
    F.dayofweek('event_date').isin([1,7])
    )\
    .groupBy('is_weekend',"event_type")\
    .count()
events_new.show()
# Conversion rate for weekend vs weekday
events_conversion = events_new.groupBy('is_weekend').pivot('event_type').sum('count')
events_conversion=events_conversion.withColumn('conversion_rate', (F.col('purchase')/F.col('view'))*100)
events_conversion.show()

weekend_rate = events_conversion.filter(F.col("is_weekend") == True).select("conversion_rate").first()[0]
weekday_rate = events_conversion.filter(F.col("is_weekend") == False).select("conversion_rate").first()[0]

# Compare
if weekend_rate > weekday_rate:
    print("Weekend conversion rate is higher than weekday")
else:
    print("Weekday conversion rate is higher than weekend")


In [0]:
# Correlation between price and conversion rate based on product id
df_conversion=events.groupBy('product_id')\
    .agg(
        F.avg('price').alias('avg_price'),
        F.sum(F.when(F.col('event_type')=='view',1).otherwise(0)).alias('view'),
        F.sum(F.when(F.col('event_type')=='purchase',1).otherwise(0)).alias('purchase')
    )\
    .filter(F.col('view')>0)\
    .withColumn(
        'conversion_rate',
        (F.try_divide(F.col('purchase'), F.col('view')))*100
    )

result = df_conversion.corr("avg_price", "conversion_rate")
print(f'Correlation between avg_price and conversion rate',{result})

In [0]:
from pyspark.sql.window import Window
# Feature engineering
features = events.filter(F.col('price')>0)\
    .withColumn("hour", F.hour("event_time")) \
    .withColumn("day_of_week", F.dayofweek("event_date")) \
    .withColumn("price_log", F.log(F.col("price")+1)) \
    .withColumn("time_since_first_view",
        F.unix_timestamp("event_time").cast('long') -
        F.first("event_time").over(Window.partitionBy("user_id").orderBy("event_time")).cast('long'))
features.limit(5).display()
features.printSchema()